In [11]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import re
import networkx as nx
import nltk
nltk.download(['punkt','stopwords'])
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/ellie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ellie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Read in dataframe and drop any duplciates from the dataset
dataset_df = pd.read_csv(r'/Users/ellie/Desktop/dataset_to_train.csv')
dataset_df = dataset_df.drop_duplicates()

# Make all words lowercase
dataset_df["clean_tweet"]=dataset_df["tweet"].str.lower()

# Remove standard stopwords
dataset_df['clean_tweet'] = dataset_df['clean_tweet'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords]))

# Remove links/urls
def remove_url(txt):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())
all_tweets_no_urls = [remove_url(tweet) for tweet in dataset_df['clean_tweet']]
all_tweets_no_urls[:5]
dataset_df['clean_tweet'] = dataset_df['clean_tweet'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])

# Remove null values
(dataset_df.isnull().any().any())
dataset_df.info(null_counts=True)
df_new = dataset_df[dataset_df['clean_tweet'].notnull()]
df_new.info(null_counts=True)
df_new.isnull().any().any()
df_new

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1396 entries, 0 to 1395
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1396 non-null   int64 
 1   tweet        1396 non-null   object
 2   class        1396 non-null   int64 
 3   clean_tweet  1396 non-null   object
dtypes: int64(2), object(2)
memory usage: 54.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1396 entries, 0 to 1395
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1396 non-null   int64 
 1   tweet        1396 non-null   object
 2   class        1396 non-null   int64 
 3   clean_tweet  1396 non-null   object
dtypes: int64(2), object(2)
memory usage: 54.5+ KB


,Unnamed: 0,tweet,class,clean_tweet
0,0,@only4itzy :((((,0,@only4itzy :((((
1,1,@3VILCTRL DAMI,0,@3vilctrl dami
2,2,Empecemos recordando cuando se puso a bailar p...,0,empecemos recordando cuando se puso bailar por...
3,3,"@defaiite może już sobie odpocznij, to był dłu...",0,"@defaiite może już sobie odpocznij, był długi ..."
4,4,@idxle_town Thanks!,0,@idxle_town thanks!
...,...,...,...,...
1391,95,Everybody is talking about #EndSARS and this ...,1,everybody talking #endsars always tweeting.
1392,96,"@maxi9990 @BillKristol Liberals always say ""th...",1,"@maxi9990 @billkristol liberals always say ""th..."
1393,97,@AngelPrincess97 I you keep feeling better. On...,1,@angelprincess97 keep feeling better. one many...
1394,98,my weird ship is I hc bei and childe as enemie...,1,weird ship hc bei childe enemies lovers childe...


In [13]:
# Remove Usernames/twitter handles 
df_new['text'] = df_new['clean_tweet']

for i in range(len(df_new['clean_tweet'])):
    try:
        df_new['clean_tweet'][i] = df_new['text'].str.split(' ')[i][0]
    except AttributeError:    
        df_new['clean_tweet'][i] = 'other'

        
# remove URLs, RTs, and twitter handles
for i in range(len(df_new['text'])):
    df_new['text'][i] = " ".join([word for word in df_new['text'][i].split()
                                if 'http' not in word and '@' not in word and '<' not in word])


df_new['text'][1]
df_new

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,Unnamed: 0,tweet,class,clean_tweet,text
0,0,@only4itzy :((((,0,@only4itzy,:((((
1,1,@3VILCTRL DAMI,0,@3vilctrl,dami
2,2,Empecemos recordando cuando se puso a bailar p...,0,empecemos,empecemos recordando cuando se puso bailar por...
3,3,"@defaiite może już sobie odpocznij, to był dłu...",0,@defaiite,"może już sobie odpocznij, był długi dzien"
4,4,@idxle_town Thanks!,0,@idxle_town,thanks!
...,...,...,...,...,...
1391,95,Everybody is talking about #EndSARS and this ...,1,everybody,everybody talking #endsars always tweeting.
1392,96,"@maxi9990 @BillKristol Liberals always say ""th...",1,@maxi9990,"liberals always say ""the appearance fraud inve..."
1393,97,@AngelPrincess97 I you keep feeling better. On...,1,@angelprincess97,keep feeling better. one many reasons hate cov...
1394,98,my weird ship is I hc bei and childe as enemie...,1,weird,weird ship hc bei childe enemies lovers childe...


In [17]:
# Remove hashtag symbol
df_new['text'] = df_new['text'].str.replace('#', '')

In [18]:
# Export cleaned dataset to CSV file
df_new.to_csv(r'/Users/ellie/Desktop/dataset_to_train_CLEANED.csv')